<a href="https://colab.research.google.com/github/Toowiredd/AutoGPT/blob/master/Copy_of_yolov8trainingfromvideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'cdsset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5399261%2F8968850%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240811%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240811T010654Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8c47ed80b4c29ed8f1727e822297ba24c9fee0eb586f9ed6d94cf045d22a0d3186c61a16f57843739975a5625402d72caa6431848acec75f08cdc0907ea4d969d2ce8594ad8f5dd984fbe3dd98b58eb1cb04be3f2dd704571405e984b92b782c6730a19980f5c2ae9ac5d851902a95cfc211e660632ff5cf12d9953a9cb6a57943f6223428d71a29f5ae0ca713d611d0ab7be43838154a826e5436575d00d5370a98bb5d8e8d14b9fd8125546290f2081679988f8a92a5ceea8705b44cc322680e277f3ea50a1f78864f8d72041e8ee05bdb2205de263f337100bd5cb9e483c82fb1f2b995999fff4717ec52e71a79e07d208480c6fca702407deb03a70a7294,bottle-videos:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5471566%2F9071211%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240811%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240811T010654Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D77fc511f82cc155778b6de92feee22740348cddce5d02c12db03a2ab7afbe7c4170878489a6d905f61d9a21cd4d22b54b70752a010f98370a9f22cd94d676db88bc391d126efb89d39038c7db48a004e1182aad3e2d7506a85c61cf75565640ff562adafe83dc13b466cac5aca229fdcd77ff431fba550e4c1ec0df48f3d446f9ca889eafe534e9f20921826446c0afbe8b51c2d4244603d2e719f97be34e852e0887154ea46b8ba8467d60ff99f84c29ab825a8b9294b0f396cece06252f74f275871aeee8297d500541ad5f0cf82df6aeca0c2a15d70fb340e2b72c2283dee68472f2f18e9fa21a71e05a47b37cb7acb2c05b09b40c682a23cfee2ded775b7,yolov8/tflite/yolov8-bottlesncans/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F74641%2F88967%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240811%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240811T010654Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6e043ca2bb967f7a10489d585a2ffc280ea1a17bbe06a3e406eaa52044d8f0a0874c45a3f08f4ef2ae38357aefb08d07be5a35d2e115f0e8a36a997cf225bdcda1addc91236a1231690958c9de91ac0bfeb0da563edc33f45b7199a2800b7e24cf4147e99edada51552a01254c5ad90a5c239064dbd527e157e2b1c005eed15bcc7b8d93e5a11606c19e5c1b177d472d2cd396cb7db1ddbdfe9b409a1e73d0bc576822c46477b97da31d698db01f7b328faf48cf6e8e0bfc97a5e93c17f62851606eb6f47b4159964e69b967e708dd0b64137d40725615ed5213388a45cdf795a8aeb67d2fbd3b92176ae241bc4da1499ef6a05ab051018d6374cb2e9e7a9d68'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


TRAINING YOLOV8 FOR DRINK CONTAINER DETECTI0N

Set up the environment and import libraries


In [ ]:
!pip install ultralytics
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
import torchvision
import cv2
import numpy as np
import os
import shutil
from PIL import Image, ImageDraw, ImageFont
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.transforms import functional as F
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import logging
import argparse
from ultralytics import YOLO
import albumentations as A
import yaml
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

Model Initialization





In [ ]:
def get_object_detection_model(local_rank, model_type='yolov8'):
    device = torch.device(f"cuda:{local_rank}" if torch.cuda.is_available() else "cpu")

    if model_type == 'yolov8':
        model = YOLO('yolov8n.pt')  # Load a pretrained YOLOv8 model
    else:
        # Fallback to Faster R-CNN
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    model = model.to(device)

    if torch.cuda.device_count() > 1:
        model = DDP(model, device_ids=[local_rank])

    return model, model.names if model_type == 'yolov8' else FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT.meta["categories"]

Data Augmentation

In [ ]:
def get_augmentation():
    return A.Compose([
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.5),
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Blur(blur_limit=3, p=0.3),
    ], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

augmentation = get_augmentation()

Video Processing with Augmentation

In [ ]:
def process_video(video_file, model, categories, user_labels, base_dir, local_rank, max_frames):
    device = torch.device(f"cuda:{local_rank}" if torch.cuda.is_available() else "cpu")
    cap = cv2.VideoCapture(video_file)

    frame_count = 0
    while cap.isOpened() and (max_frames is None or frame_count < max_frames):
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame_rgb)

        # Detect objects
        if isinstance(model, YOLO):
            results = model(frame_rgb)
            boxes = results[0].boxes.xyxy.cpu().numpy()
            labels = results[0].boxes.cls.cpu().numpy()
            scores = results[0].boxes.conf.cpu().numpy()
        else:
            tensor = F.to_tensor(image).unsqueeze(0).to(device)
            with torch.no_grad():
                prediction = model(tensor)[0]
            boxes = prediction['boxes'].cpu().numpy()
            labels = prediction['labels'].cpu().numpy()
            scores = prediction['scores'].cpu().numpy()

        # Apply augmentation
        augmented = augmentation(image=np.array(image), bboxes=boxes, class_labels=labels)
        augmented_image = Image.fromarray(augmented['image'])
        augmented_boxes = np.array(augmented['bboxes'])
        augmented_labels = np.array(augmented['class_labels'])

        # Label and save original objects
        label_and_save_objects(image, boxes, labels, scores, categories, user_labels, base_dir, frame_count, "original")

        # Label and save augmented objects
        label_and_save_objects(augmented_image, augmented_boxes, augmented_labels, scores, categories, user_labels, base_dir, frame_count, "augmented")

        frame_count += 1

    cap.release()
    return frame_count

Create YAML Configuration

In [ ]:
def create_yaml_config(base_dir, class_names):
    config = {
        'path': base_dir,
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'nc': len(class_names),
        'names': class_names
    }

    with open(os.path.join(base_dir, 'dataset.yaml'), 'w') as f:
        yaml.dump(config, f)

    logging.info(f"Created YAML configuration at {os.path.join(base_dir, 'dataset.yaml')}")

Split Dataset

In [ ]:
def split_dataset(base_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    image_dir = os.path.join(base_dir, 'images')
    label_dir = os.path.join(base_dir, 'labels')

    all_images = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]

    train_images, temp_images = train_test_split(all_images, train_size=train_ratio, random_state=42)
    val_images, test_images = train_test_split(temp_images, train_size=val_ratio/(val_ratio+test_ratio), random_state=42)

    for split, images in [('train', train_images), ('val', val_images), ('test', test_images)]:
        os.makedirs(os.path.join(image_dir, split), exist_ok=True)
        os.makedirs(os.path.join(label_dir, split), exist_ok=True)
        for img in images:
            shutil.move(os.path.join(image_dir, img), os.path.join(image_dir, split, img))
            label = img.rsplit('.', 1)[0] + '.txt'
            if os.path.exists(os.path.join(label_dir, label)):
                shutil.move(os.path.join(label_dir, label), os.path.join(label_dir, split, label))

    logging.info(f"Dataset split: {len(train_images)} train, {len(val_images)} validation, {len(test_images)} test")

Create directories for labeled data


In [ ]:
def create_directories():
    base_dir = "/kaggle/working/labeled_data"
    dirs = [
        f"{base_dir}/images/train",
        f"{base_dir}/images/val",
        f"{base_dir}/labels/train",
        f"{base_dir}/labels/val"
    ]
    for dir in dirs:
        os.makedirs(dir, exist_ok=True)
    return base_dir

base_dir = create_directories()

Visualize Labeled Images

In [ ]:
def visualize_labeled_images(base_dir, num_images=5):
    image_dir = os.path.join(base_dir, 'images', 'train')
    label_dir = os.path.join(base_dir, 'labels', 'train')

    images = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
    selected_images = np.random.choice(images, num_images, replace=False)

    fig, axs = plt.subplots(1, num_images, figsize=(20, 4))
    for i, img_name in enumerate(selected_images):
        img_path = os.path.join(image_dir, img_name)
        label_path = os.path.join(label_dir, img_name.rsplit('.', 1)[0] + '.txt')

        img = Image.open(img_path)
        draw = ImageDraw.Draw(img)

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    class_id, x, y, w, h = map(float, line.strip().split())
                    left = (x - w/2) * img.width
                    top = (y - h/2) * img.height
                    right = (x + w/2) * img.width
                    bottom = (y + h/2) * img.height
                    draw.rectangle([left, top, right, bottom], outline='red', width=2)

        axs[i].imshow(img)
        axs[i].axis('off')
        axs[i].set_title(f'Image {i+1}')

    plt.tight_layout()
    plt.show()

Main Function

In [ ]:
def main():
    parser = argparse.ArgumentParser(description="YOLOv8 Training for Drink Container Detection")
    parser.add_argument("--input_dir", type=str, required=True, help="Input directory containing video files")
    parser.add_argument("--output_dir", type=str, required=True, help="Output directory for processed data")
    parser.add_argument("--max_frames", type=int, default=None, help="Maximum number of frames to process per video")
    parser.add_argument("--num_gpus", type=int, default=1, help="Number of GPUs to use")
    args = parser.parse_args()

    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    video_files = [os.path.join(args.input_dir, f) for f in os.listdir(args.input_dir) if f.endswith(('.mp4', '.avi', '.mov'))]
    user_labels = get_user_labels()  # Implement this function to get user-defined labels

    if args.num_gpus > 1 and torch.cuda.is_available():
        world_size = min(args.num_gpus, torch.cuda.device_count())
        mp.spawn(distributed_process_videos,
                 args=(world_size, video_files, user_labels, args.output_dir, args.max_frames),
                 nprocs=world_size,
                 join=True)
    else:
        model, categories = get_object_detection_model(0)
        for video_file in tqdm(video_files, desc="Processing videos"):
            process_video(video_file, model, categories, user_labels, args.output_dir, 0, args.max_frames)

    # Split the dataset
    split_dataset(args.output_dir)

    # Create YAML configuration
    create_yaml_config(args.output_dir, list(user_labels.keys()))

    # Visualize some labeled images
    visualize_labeled_images(args.output_dir)

    logging.info("Video processing completed. Dataset created at %s", args.output_dir)

if __name__ == "__main__":
    main()

usage: colab_kernel_launcher.py [-h] --input_dir INPUT_DIR --output_dir OUTPUT_DIR
                                [--max_frames MAX_FRAMES] [--num_gpus NUM_GPUS]
colab_kernel_launcher.py: error: the following arguments are required: --input_dir, --output_dir


SystemExit: 2

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Load and preprocess the video


In [ ]:

#def load_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file at {video_path}")
        return []

    frames = []
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count} frames")

 #   cap.release()
 #   print(f"Total frames processed: {len(frames)}")
#    return frames

#video_path = "/kaggle/input/bottle-videos" # Make sure this path is correct
#frames = load_video(video_path)

IndentationError: unexpected indent (4283674236.py, line 2)

Implement object detection

In [ ]:
def process_video(video_file, model, categories, user_labels, base_dir, local_rank, max_frames):
    device = torch.device(f"cuda:{local_rank}" if torch.cuda.is_available() else "cpu")


In [ ]:
def get_object_detection_model():
    weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
    model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
    model.eval()
    return model, weights.meta["categories"]

model, categories = get_object_detection_model()

def detect_objects(image, model):
    tensor = F.to_tensor(image).unsqueeze(0)
    with torch.no_grad():
        prediction = model(tensor)[0]
    return prediction

Label detected objects and save data

In [ ]:
def label_and_save_objects(image, prediction, categories, user_labels, base_dir, idx, split):
    img = Image.fromarray(image)
    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 20)

    img_path = f"{base_dir}/images/{split}/frame_{idx:04d}.jpg"
    label_path = f"{base_dir}/labels/{split}/frame_{idx:04d}.txt"

    img.save(img_path)

    with open(label_path, 'w') as f:
        for box, label, score in zip(prediction["boxes"], prediction["labels"], prediction["scores"]):
            if score > 0.5:
                box = box.tolist()
                category = categories[label]
                if category in user_labels:
                    color = user_labels[category]
                    draw.rectangle(box, outline=color, width=3)
                    draw.text((box[0], box[1]), f"{category}: {score:.2f}", fill=color, font=font)

                    # Convert box coordinates to YOLO format
                    x_center = (box[0] + box[2]) / 2 / img.width
                    y_center = (box[1] + box[3]) / 2 / img.height
                    width = (box[2] - box[0]) / img.width
                    height = (box[3] - box[1]) / img.height

                    # Write label in YOLO format
                    f.write(f"{list(user_labels.keys()).index(category)} {x_center} {y_center} {width} {height}\n")

    return np.array(img)

# Get user input for labels and colors
user_labels = {}
while True:
    category = input("Enter a category to label (or 'done' to finish): ")
    if category.lower() == 'done':
        break
    color = input(f"Enter a color for {category} (e.g., 'red', 'blue', 'green'): ")
    user_labels[category] = color

Enter a category to label (or 'done' to finish):  PLASTIC BOTTLES
Enter a color for PLASTIC BOTTLES (e.g., 'red', 'blue', 'green'):  RED
Enter a category to label (or 'done' to finish):  GLASS BOTTLES
Enter a color for GLASS BOTTLES (e.g., 'red', 'blue', 'green'):  BLUE
Enter a category to label (or 'done' to finish):  MILK CARTONS
Enter a color for MILK CARTONS (e.g., 'red', 'blue', 'green'):  GREEN
Enter a category to label (or 'done' to finish):  DONE


Process video and create dataset


In [ ]:
def process_video_and_create_dataset(frames, model, categories, user_labels, base_dir):
    processed_frames = []

    # Split frames into train and validation sets
    train_frames, val_frames = train_test_split(frames, test_size=0.2, random_state=42)

    for split, split_frames in [("train", train_frames), ("val", val_frames)]:
        for idx, frame in enumerate(split_frames):
            prediction = detect_objects(frame, model)
            labeled_frame = label_and_save_objects(frame, prediction, categories, user_labels, base_dir, idx, split)
            processed_frames.append(labeled_frame)

    return processed_frames

processed_frames = process_video_and_create_dataset(frames, model, categories, user_labels, base_dir)

NameError: name 'base_dir' is not defined

Display results and create dataset info


In [ ]:
# Display the first processed frame
from IPython.display import Image as IPImage
from IPython.display import display

first_frame = Image.fromarray(processed_frames[0])
first_frame.save("first_frame.jpg")
display(IPImage("first_frame.jpg"))

# Create dataset info file
def create_dataset_info(base_dir, user_labels):
    with open(f"{base_dir}/dataset_info.txt", 'w') as f:
        f.write("Dataset Information\n")
        f.write("-------------------\n")
        f.write(f"Total frames: {len(processed_frames)}\n")
        f.write(f"Training frames: {len(os.listdir(f'{base_dir}/images/train'))}\n")
        f.write(f"Validation frames: {len(os.listdir(f'{base_dir}/images/val'))}\n")
        f.write("\nLabels:\n")
        for idx, (category, color) in enumerate(user_labels.items()):
            f.write(f"{idx}: {category} ({color})\n")

create_dataset_info(base_dir, user_labels)

print(f"Labeled dataset created at {base_dir}")
print(f"Dataset information saved to {base_dir}/dataset_info.txt")

NameError: name 'processed_frames' is not defined

In [ ]:
to use this updated notebook now includes the following additional features:

Creates separate directories for images and labels, both for training and validation sets.
Splits the video frames into training and validation sets (80% train, 20% validation).
Saves labeled images and corresponding label files in YOLO format.
Creates a dataset information file with details about the number of frames and label categories.
To use this notebook in Kaggle:

Create a new notebook in your Kaggle environment.
Copy and paste the code snippets into the appropriate cells.
Upload your video file to the Kaggle dataset and update the video_path variable.
Run the cells in order, providing input for the categories you want to label when prompted.

In [ ]:
# %% [markdown]
# # Video Processing and Object Detection Dataset Creation
#
# This notebook processes video files to create a labeled dataset for object detection. It uses the Faster R-CNN model for object detection and allows users to specify categories and colors for labeling.

# %% [markdown]
# ## Import Required Libraries

# %%
import cv2
import torch
import torchvision
import numpy as np
import os
import shutil
from PIL import Image, ImageDraw, ImageFont
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.transforms import functional as F
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed

# %% [markdown]
# ## Set up logging

# %%
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# %% [markdown]
# ## Directory Creation and File Handling

# %%
def create_directories(base_dir):
    dirs = [
        f"{base_dir}/images/train",
        f"{base_dir}/images/val",
        f"{base_dir}/labels/train",
        f"{base_dir}/labels/val"
    ]
    for dir in dirs:
        os.makedirs(dir, exist_ok=True)
    return base_dir

def get_video_files(root_dir):
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv']
    video_files = []
    for root, _, files in os.walk(root_dir):
        for file in files:
            if any(file.lower().endswith(ext) for ext in video_extensions):
                video_files.append(os.path.join(root, file))
    return video_files

# %% [markdown]
# ## Video Loading and Processing

# %%
def load_video(video_path, max_frames=None):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        logging.error(f"Could not open video file at {video_path}")
        return []

    frames = []
    frame_count = 0
    with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), desc=f"Loading {os.path.basename(video_path)}") as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or (max_frames and frame_count >= max_frames):
                break
            frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            frame_count += 1
            pbar.update(1)

    cap.release()
    logging.info(f"Loaded {len(frames)} frames from {video_path}")
    return frames

# %% [markdown]
# ## Object Detection Model

# %%
def get_object_detection_model(device):
    weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
    model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
    model.to(device)
    model.eval()
    return model, weights.meta["categories"]

def detect_objects(image, model, device):
    tensor = F.to_tensor(image).unsqueeze(0).to(device)
    with torch.no_grad():
        prediction = model(tensor)[0]
    return {k: v.cpu() for k, v in prediction.items()}

# %% [markdown]
# ## Object Labeling and Saving

# %%
def label_and_save_objects(image, prediction, categories, user_labels, base_dir, idx, split):
    img = Image.fromarray(image)
    draw = ImageDraw.Draw(img)
    font = ImageFont.load_default()

    img_path = f"{base_dir}/images/{split}/frame_{idx:06d}.jpg"
    label_path = f"{base_dir}/labels/{split}/frame_{idx:06d}.txt"

    img.save(img_path)

    with open(label_path, 'w') as f:
        for box, label, score in zip(prediction["boxes"], prediction["labels"], prediction["scores"]):
            if score > 0.5:
                box = box.tolist()
                category = categories[label]
                if category in user_labels:
                    color = user_labels[category]
                    draw.rectangle(box, outline=color, width=3)
                    draw.text((box[0], box[1]), f"{category}: {score:.2f}", fill=color, font=font)

                    x_center = (box[0] + box[2]) / 2 / img.width
                    y_center = (box[1] + box[3]) / 2 / img.height
                    width = (box[2] - box[0]) / img.width
                    height = (box[3] - box[1]) / img.height

                    f.write(f"{list(user_labels.keys()).index(category)} {x_center} {y_center} {width} {height}\n")

    return np.array(img)

# %% [markdown]
# ## Video Processing Functions

# %%
def process_video(video_file, model, categories, user_labels, base_dir, device, max_frames):
    frames = load_video(video_file, max_frames)
    processed_frames = []

    for idx, frame in enumerate(tqdm(frames, desc=f"Processing {os.path.basename(video_file)}")):
        prediction = detect_objects(frame, model, device)
        labeled_frame = label_and_save_objects(frame, prediction, categories, user_labels, base_dir, idx, "train")
        processed_frames.append(labeled_frame)

    return processed_frames

def process_videos_and_create_dataset(video_files, model, categories, user_labels, base_dir, device, max_frames, num_workers):
    all_processed_frames = []

    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        future_to_video = {executor.submit(process_video, video, model, categories, user_labels, base_dir, device, max_frames): video for video in video_files}
        for future in tqdm(as_completed(future_to_video), total=len(video_files), desc="Processing videos"):
            video = future_to_video[future]
            try:
                processed_frames = future.result()
                all_processed_frames.extend(processed_frames)
            except Exception as exc:
                logging.error(f"{video} generated an exception: {exc}")

    # Split frames into train and validation sets
    train_frames, val_frames = train_test_split(all_processed_frames, test_size=0.2, random_state=42)

    # Move validation frames to val directory
    for idx, frame in enumerate(val_frames):
        src_img = f"{base_dir}/images/train/frame_{idx:06d}.jpg"
        src_label = f"{base_dir}/labels/train/frame_{idx:06d}.txt"
        dst_img = f"{base_dir}/images/val/frame_{idx:06d}.jpg"
        dst_label = f"{base_dir}/labels/val/frame_{idx:06d}.txt"

        shutil.move(src_img, dst_img)
        shutil.move(src_label, dst_label)

    return all_processed_frames

# %% [markdown]
# ## Dataset Information Creation

# %%
def create_dataset_info(base_dir, user_labels, processed_frames):
    info = {
        "total_frames": len(processed_frames),
        "training_frames": len(os.listdir(f'{base_dir}/images/train')),
        "validation_frames": len(os.listdir(f'{base_dir}/images/val')),
        "labels": {idx: {"category": category, "color": color} for idx, (category, color) in enumerate(user_labels.items())}
    }

    with open(f"{base_dir}/dataset_info.json", 'w') as f:
        json.dump(info, f, indent=4)

    logging.info(f"Dataset information saved to {base_dir}/dataset_info.json")

# %% [markdown]
# ## Main Execution

# %%
def main(input_dir, output_dir, max_frames=None, num_workers=4):
    base_dir = create_directories(output_dir)
    video_files = get_video_files(input_dir)

    if not video_files:
        logging.error("No video files found in the specified directory and its subdirectories.")
        return

    logging.info(f"Found {len(video_files)} video files.")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model, categories = get_object_detection_model(device)

    user_labels = {}
    while True:
        category = input("Enter a category to label (or 'done' to finish): ")
        if category.lower() == 'done':
            break
        color = input(f"Enter a color for {category} (e.g., 'red', 'blue', 'green'): ")
        user_labels[category] = color

    processed_frames = process_videos_and_create_dataset(video_files, model, categories, user_labels, base_dir, device, max_frames, num_workers)

    if processed_frames:
        create_dataset_info(base_dir, user_labels, processed_frames)
        logging.info(f"Labeled dataset created at {base_dir}")
    else:
        logging.warning("No frames were processed. Check if the video files are valid and contain frames.")

# %% [markdown]
# ## Run the Script
#
# To run the script, set the `input_dir` and `output_dir` variables below, and then execute this cell.

# %%
input_dir = "/kaggle/input/cdsset"  # Replace with your input directory
input_dir = "/kaggle/input/bottle-videos"
output_dir = "/kaggle/working/"  # Replace with your output directory
max_frames = None # Set to None to process all frames
num_workers = 4  # Adjust based on your system's capabilities

main(input_dir, output_dir, max_frames, num_workers)

# %% [markdown]
# ## Conclusion
#
# This notebook processes video files to create a labeled dataset for object detection. It uses the Faster R-CNN model for object detection and allows users to specify categories and colors for labeling. The resulting dataset is split into training and validation sets, and a JSON file with dataset information is created.

ERROR:root:No video files found in the specified directory and its subdirectories.
